# Pre-Training Validation

Validate dataset before training to ensure quality and format correctness.

In [1]:
import sys
from pathlib import Path
import json
import random
from datetime import datetime

sys.path.append(str(Path('../utils/validators').resolve()))
from schema_validator import SchemaValidator, validate_preferences_file

ModuleNotFoundError: No module named 'jsonschema'

## Configuration

In [ ]:
DATASET_PATH = Path('../data/training_dataset/synthetic_preferences.jsonl')
CONFIG_PATH = Path('../config/training_config.json')

with open(CONFIG_PATH, 'r') as f:
    training_config = json.load(f)

print("Training Configuration:")
print(f"  DPO beta: {training_config['dpo_config']['beta']}")
print(f"  Learning rate: {training_config['dpo_config']['learning_rate']}")
print(f"  Batch size: {training_config['dpo_config']['batch_size']}")
print(f"  Epochs: {training_config['dpo_config']['num_epochs']}")

## Format Validation

In [ ]:
if DATASET_PATH.exists():
    result = validate_preferences_file(DATASET_PATH)
    print("Format Validation Results:")
    print("=" * 50)
    print(f"  File: {result['file']}")
    print(f"  Total: {result.get('total', 0)}")
    print(f"  Valid: {result.get('valid', 0)}")
    print(f"  Invalid: {result.get('invalid', 0)}")
    
    if result.get('errors'):
        print(f"\nErrors ({len(result['errors'])}):")
        for err in result['errors'][:3]:
            print(f"  Item {err['index']}: {err['errors'][:2]}")
    else:
        print("\nAll examples passed validation")
else:
    print(f"Dataset not found at {DATASET_PATH}")

## Sample Review

In [ ]:
def load_samples(filepath: Path, n: int = 3) -> list:
    examples = []
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip():
                examples.append(json.loads(line))
    random.seed(42)
    return random.sample(examples, min(n, len(examples)))

if DATASET_PATH.exists():
    samples = load_samples(DATASET_PATH, n=3)
    print("Sample Examples:")
    print("=" * 60)
    for i, ex in enumerate(samples):
        print(f"\n--- Example {i+1} ---")
        print(f"Source: {ex.get('source', 'N/A')}")
        print(f"Weight: {ex.get('weight', 'N/A')}")
        pref = ex.get('preferred', {})
        print(f"Preferred: {pref.get('description', 'N/A')[:60]}...")
        print(f"  Confidence: {pref.get('confidence', 'N/A')}")

## Data Loading Test

In [ ]:
def test_data_loading(filepath: Path):
    examples = []
    errors = []
    with open(filepath, 'r') as f:
        for line_num, line in enumerate(f, 1):
            if line.strip():
                try:
                    ex = json.loads(line)
                    assert 'preferred' in ex
                    assert 'rejected' in ex
                    examples.append(ex)
                except Exception as e:
                    errors.append((line_num, str(e)))
    return examples, errors

if DATASET_PATH.exists():
    examples, errors = test_data_loading(DATASET_PATH)
    print("Data Loading Test:")
    print("=" * 50)
    print(f"  Loaded: {len(examples)} examples")
    print(f"  Errors: {len(errors)}")
    if errors:
        for ln, err in errors[:3]:
            print(f"    Line {ln}: {err}")

## Cost Estimation

In [ ]:
if DATASET_PATH.exists() and examples:
    num_examples = len(examples)
    batch_size = training_config['dpo_config']['batch_size']
    grad_accum = training_config['dpo_config']['gradient_accumulation_steps']
    num_epochs = training_config['dpo_config']['num_epochs']
    
    effective_batch = batch_size * grad_accum
    steps_per_epoch = num_examples // effective_batch
    total_steps = steps_per_epoch * num_epochs
    est_minutes = total_steps / 60
    
    print("Training Cost Estimation:")
    print("=" * 50)
    print(f"  Dataset size:     {num_examples}")
    print(f"  Effective batch:  {effective_batch}")
    print(f"  Steps per epoch:  {steps_per_epoch}")
    print(f"  Total steps:      {total_steps}")
    print(f"  Est. time:        ~{est_minutes:.0f} min (GPU)")

## Data Requirements

In [ ]:
if DATASET_PATH.exists() and examples:
    min_rec = training_config['data_config']['min_samples']
    optimal = training_config['data_config']['optimal_samples']
    
    print("Data Requirements:")
    print("=" * 50)
    if num_examples >= optimal:
        print(f"  Status: OPTIMAL")
    elif num_examples >= min_rec:
        print(f"  Status: MINIMUM MET")
    else:
        print(f"  Status: BELOW MINIMUM")
    print(f"  Current: {num_examples}")
    print(f"  Minimum: {min_rec}")
    print(f"  Optimal: {optimal}")

## Train/Val/Test Split

In [ ]:
if DATASET_PATH.exists() and examples:
    train_split = training_config['data_config']['train_split']
    val_split = training_config['data_config']['val_split']
    test_split = training_config['data_config']['test_split']
    
    train_size = int(num_examples * train_split)
    val_size = int(num_examples * val_split)
    test_size = num_examples - train_size - val_size
    
    print("Train/Val/Test Split:")
    print("=" * 50)
    print(f"  Train: {train_size} ({train_split*100:.0f}%)")
    print(f"  Val:   {val_size} ({val_split*100:.0f}%)")
    print(f"  Test:  {test_size} ({test_split*100:.0f}%)")

## Final Checklist

In [ ]:
print("Pre-Training Checklist:")
print("=" * 50)

checks = []
checks.append(("Dataset file exists", DATASET_PATH.exists()))

if DATASET_PATH.exists():
    result = validate_preferences_file(DATASET_PATH)
    checks.append(("Format validation passed", result.get('invalid', 0) == 0))
    _, load_errors = test_data_loading(DATASET_PATH)
    checks.append(("Data loading works", len(load_errors) == 0))
    checks.append(("Minimum data size", len(examples) >= min_rec))
else:
    checks.append(("Format validation passed", False))
    checks.append(("Data loading works", False))
    checks.append(("Minimum data size", False))

checks.append(("Training config exists", CONFIG_PATH.exists()))

for name, passed in checks:
    status = "PASS" if passed else "FAIL"
    print(f"  [{status}] {name}")

all_passed = all(p for _, p in checks)
print(f"\n{'Ready for training!' if all_passed else 'Fix issues before training'}")

## Next Steps

If all checks pass, proceed to DPO training with:
```python
from training.dpo_trainer import DPOTrainer
trainer = DPOTrainer(config_path='config/training_config.json')
trainer.train(dataset_path='data/training_dataset/synthetic_preferences.jsonl')
```